# Spark MLlib - LinearRegression example

<p>Obs.: After download the databricks notebook to .ipynb we have problems in the output format but if you run this notebook in a databricks cluster you'll have a output in a table format.</p>

<p>E.g.:</p>
<p>The following output:</p>
<p>+----+-------+ age| name| +----+-------+ null|Michael| 30| Andy| 19| Justin| +----+-------+</p>
<p>actually is:</p>
<pre>+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+  </pre>

### Create session

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

### Import LinearRegression

In [3]:
from pyspark.ml.regression import LinearRegression

### Load csv data

In [4]:
data = spark.read.csv('/FileStore/tables/Ecommerce_Customers-83b1c.csv', header=True, inferSchema=True)

In [5]:
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [6]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
 Email| Address| Avatar|Avg Session Length| Time on App| Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
mstephenson@ferna...|835 Frank TunnelW...| Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616| 4.0826206329529615| 587.9510539684005|
 hduke@hotmail.com|4547 Archer Commo...| DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744| 2.66403418213262| 392.2049334443264|
 pallen@yahoo.com|24645 Valerie Uni...| Bisque|33.000914755642675|11.330278057777512|37.110597442120856| 4.104543202376424| 487.54750486747207|
riverarebecca@gma...|1414 David Throug...| SaddleBrown| 34.30555662975554|13.717513665142507| 36.72128267790313| 3.120178782748092| 581.8523440352177|
mstephens@davidso...|14023 Rodriguez P...|MediumAquaMarine| 33.33067252364639|12.795188551078114| 37.53665330059473| 4.446308318351434| 599.4060920457634|
alvareznancy@luca...|645 Martha Park A...| FloralWhite|33.871037879341976|12.026925339755056| 34.47687762925054| 5.493507201364199| 637.102447915074|
katherine20@yahoo...|68388 Reyes Light...| DarkSlateBlue| 32.02159550138701|11.366348309710526| 36.68377615286961| 4.685017246570912| 521.5721747578274|
 awatkins@yahoo.com|Unit 6538 Box 898...| Aqua|32.739142938380326| 12.35195897300293| 37.37335885854755| 4.4342734348999375| 549.9041461052942|
vchurch@walter-ma...|860 Lee KeyWest D...| Salmon| 33.98777289568564|13.386235275676436|37.534497341555735| 3.2734335777477144| 570.2004089636196|
 bonnie69@lin.biz|PSC 2734, Box 525...| Brown|31.936548618448917|11.814128294972196| 37.14516822352819| 3.202806071553459| 427.1993848953282|
andrew06@peterson...|26104 Alexander G...| Tomato|33.992572774953736|13.338975447662113| 37.22580613162114| 2.482607770510596| 492.6060127179966|
ryanwerner@freema...|Unit 2413 Box 034...| Tomato| 33.87936082480498|11.584782999535266| 37.08792607098381| 3.71320920294043| 522.3374046069357|
 knelson@gmail.com|6705 Miller Orcha...| RoyalBlue|29.532428967057943|10.961298400154098| 37.42021557502538| 4.046423164299585| 408.6403510726275|
wrightpeter@yahoo...|05302 Dunlap Ferr...| Bisque| 33.19033404372265|12.959226091609382|36.144666700041924| 3.918541839158999| 573.4158673313865|
taylormason@gmail...|7773 Powell Sprin...| DarkBlue|32.387975853153876|13.148725692056516| 36.61995708279922| 2.494543646659249| 470.4527333009554|
 jstark@anderson.com|49558 Ramirez Roa...| Peru|30.737720372628182|12.636606052000127|36.213763093698624| 3.3578468423262944| 461.7807421962299|
 wjennings@gmail.com|6362 Wilson Mount...| PowderBlue| 32.12538689728784|11.733861690857394| 34.8940927514398| 3.1361327164897803| 457.84769594494855|
rebecca45@hale-ba...|8982 Burton RowWi...| OliveDrab|32.338899323067196|12.013194694014402| 38.38513659413844| 2.420806160901484| 407.70454754954415|
alejandro75@hotma...|64475 Andre Club ...| Cyan|32.187812045932155| 14.7153875441565| 38.24411459434352| 1.516575580831944| 452.3156754800354|
samuel46@love-wes...|544 Alexander Hei...| LightSeaGreen| 32.61785606282345|13.989592555825254|37.190503800397956| 4.064548550437977| 605.061038804892|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
only showing top 20 rows

In [7]:
data.head(1)

Out[ 8 ]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

### Load VectorAssembler to transform the data to MLlib format

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

Out[ 11 ]: 
['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

### Instantiate assembler

In [10]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                            outputCol='features')

### Transform the data

In [11]:
output = assembler.transform(data)

In [12]:
output.select('features').show()

+--------------------+
 features|
+--------------------+
[34.4972677251122...|
[31.9262720263601...|
[33.0009147556426...|
[34.3055566297555...|
[33.3306725236463...|
[33.8710378793419...|
[32.0215955013870...|
[32.7391429383803...|
[33.9877728956856...|
[31.9365486184489...|
[33.9925727749537...|
[33.8793608248049...|
[29.5324289670579...|
[33.1903340437226...|
[32.3879758531538...|
[30.7377203726281...|
[32.1253868972878...|
[32.3388993230671...|
[32.1878120459321...|
[32.6178560628234...|
+--------------------+
only showing top 20 rows

In [13]:
output.head(1)

Out[ 16 ]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [14]:
final_data = output.select(['features', 'Yearly Amount Spent'])

In [15]:
final_data.show()

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
[33.0009147556426...| 487.54750486747207|
[34.3055566297555...| 581.8523440352177|
[33.3306725236463...| 599.4060920457634|
[33.8710378793419...| 637.102447915074|
[32.0215955013870...| 521.5721747578274|
[32.7391429383803...| 549.9041461052942|
[33.9877728956856...| 570.2004089636196|
[31.9365486184489...| 427.1993848953282|
[33.9925727749537...| 492.6060127179966|
[33.8793608248049...| 522.3374046069357|
[29.5324289670579...| 408.6403510726275|
[33.1903340437226...| 573.4158673313865|
[32.3879758531538...| 470.4527333009554|
[30.7377203726281...| 461.7807421962299|
[32.1253868972878...| 457.84769594494855|
[32.3388993230671...| 407.70454754954415|
[32.1878120459321...| 452.3156754800354|
[32.6178560628234...| 605.061038804892|
+--------------------+-------------------+
only showing top 20 rows

### Split the data

In [16]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 329|
 mean| 499.4473453295938|
 stddev| 77.58212191421666|
 min| 275.9184206503857|
 max| 765.5184619388373|
+-------+-------------------+

In [18]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 171|
 mean| 499.05755857227535|
 stddev| 82.77830300177187|
 min| 256.67058229005585|
 max| 712.3963268096637|
+-------+-------------------+

### Instantiate the model

In [19]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

### Fit the model in the training data

In [20]:
lr_model = lr.fit(train_data)

### Evaluate the model

In [21]:
test_results = lr_model.evaluate(test_data)

In [22]:
test_results.residuals.show()

+-------------------+
 residuals|
+-------------------+
 10.982035919806094|
 -18.17676706095142|
-3.7075431143638866|
-3.3140433747402653|
 7.61201295606179|
 -6.423690402545162|
 9.700105832713007|
-1.1724262179214975|
 23.412908011206582|
 18.716497360876303|
 4.392259060701122|
 -3.393083427540205|
 2.9036396724329165|
 -9.543309000730176|
 2.987448738912974|
 6.459112681703061|
-2.6862236474111683|
-17.556085411066817|
 7.269241694610116|
-10.748215457587207|
+-------------------+
only showing top 20 rows

In [23]:
test_results.rootMeanSquaredError

Out[ 28 ]: 10.318370575482522

In [24]:
test_results.r2

Out[ 29 ]: 0.9843708127824936

In [25]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [26]:
unlabeled_data = test_data.select('features')

In [27]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[29.5324289670579...|
[30.8162006488763...|
[30.8364326747734...|
[30.8794843441274...|
[30.9716756438877...|
[31.1280900496166...|
[31.1695067987115...|
[31.2606468698795...|
[31.2834474760581...|
[31.3123495994443...|
[31.3584771924370...|
[31.4252268808548...|
[31.4459724827577...|
[31.5261978982398...|
[31.5316044825729...|
[31.6548096756927...|
[31.7216523605090...|
[31.8164283341993...|
[31.8209982016720...|
[31.8648325480987...|
+--------------------+
only showing top 20 rows

### Make predictions

In [28]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[29.5324289670579...| 397.6583151528214|
[30.8162006488763...| 284.2631080094204|
[30.8364326747734...| 471.2094435413535|
[30.8794843441274...|493.52064335959494|
[30.9716756438877...|487.02659680083093|
[31.1280900496166...| 563.6763771495998|
[31.1695067987115...| 417.6564249695798|
[31.2606468698795...|422.49905747487287|
[31.2834474760581...| 568.3681814144609|
[31.3123495994443...| 444.8749206670643|
[31.3584771924370...| 490.7836913887743|
[31.4252268808548...| 534.1598020823021|
[31.4459724827577...|481.97332526269565|
[31.5261978982398...| 418.637835193068|
[31.5316044825729...| 433.5281569904496|
[31.6548096756927...|468.80431104584545|
[31.7216523605090...| 350.4631502792838|
[31.8164283341993...| 518.6785769147232|
[31.8209982016720...|417.40603931860323|
[31.8648325480987...| 450.6394959344009|
+--------------------+------------------+
only showing top 20 rows